# Assignment 1

In [4]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa

--2019-09-28 17:20:19--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.224.208.76, 13.224.208.48, 13.224.208.231, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.224.208.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49270 (48K) [application/octet-stream]
Saving to: 'lambda_virus.fa.1'

     0K .......... .......... .......... .......... ........  100% 1.44M=0.03s

2019-09-28 17:20:19 (1.44 MB/s) - 'lambda_virus.fa.1' saved [49270/49270]



In [1]:
def get_file(filename):
    out = ""
    with open(filename, "r") as file:
        for line in file:
            if ">" in line:
                continue
            out += line.strip()
    return out


dna = get_file("lambda_virus.fa")

In [3]:
def reverseComplement(s):
    complement = {"A": "T", "C": "G", "G": "C", "T": "A", "N": "N"}
    t = ""
    for base in s:
        t = complement[base] + t
    return t


def substring_match(p, t):
    for j in range(len(p)):
        if t[0 + j] != p[j]:
            return False
    return True


def exact_match(p, t):
    """
    look for occurences of p in t
    also look in the strings complement
    """
    rev = reverseComplement(p)
    matched = True if rev == p else False
    print(matched)
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = substring_match(p, t[i:])
        if match:
            print("matched")
            occurrences.append(i)
        if not matched:
            checkreverse = substring_match(rev, t[i:])
            if checkreverse:
                print("reverse matched")
                occurrences.append(i)

    return occurrences


def recurse_exact_match(pat, revpat, text, count=0):
    """
    recursive implementation of the above
    very inefficient
    """
    something = []
    if len(pat) == len(text):
        return []
    if (text[0 : len(pat)] != pat) and (text[0 : len(revpat)] != revpat):
        count += 1
        return recurse_exact_match(pat, revpat, text[1:], count)
    if (text[0 : len(pat)] == pat) or (text[0 : len(revpat)] == revpat):
        something.append(count)
        count += 1
        return something + recurse_exact_match(pat, revpat, text[1:], count)


def confirm_match(indices, dna, p):
    truth_vals = []
    for idx in indices:
        check = dna[idx : idx + len(p)]
        if check == p or check == reverseComplement(p):
            truth_vals.append(True)
        else:
            truth_vals.append(False)
    return all(x == True for x in truth_vals)

In [4]:
t = 'AGCTTAGATAGC'
p = "AG"
revpat = reverseComplement(p)
#recurse_exact_match(p,revpat,dna)

In [5]:
exact_match(p,t)

False
matched
reverse matched
matched
matched


[0, 2, 5, 9]

In [6]:
confirm_match(recurse_exact_match(p,revpat,t),t,p) == confirm_match(exact_match(p,t),t,p)

False
matched
reverse matched
matched
matched


True

In [169]:
def naive_2mm(p, t):
    """
    do the same as above, but allows two mismatches per occurence
    """
    output = []
    for i in range(len(t) - len(p) + 1):
        temp = []
        for j in range(len(p)):
            temp.append(t[i + j] == p[j])
        if temp.count(False) > 2:
            continue
        else:
            output.append(i)
    return output, output[0]

In [170]:
naive_2mm('AGGAGGTT',dna)[1]

49

In [26]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq

--2019-09-28 17:45:25--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 99.84.222.56, 99.84.222.223, 99.84.222.231, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|99.84.222.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241626 (236K) [application/octet-stream]
Saving to: 'ERR037900_1.first1000.fastq'

     0K .......... .......... .......... .......... .......... 21% 1.33M 0s
    50K .......... .......... .......... .......... .......... 42% 2.53M 0s
   100K .......... .......... .......... .......... .......... 63% 2.09M 0s
   150K .......... .......... .......... .......... .......... 84% 2.79M 0s
   200K .......... .......... .......... .....                100% 2.09M=0.1s

2019-09-28 17:45:26 (2.03 MB/s) - 'ERR037900_1.first1000.fastq' saved [241626/241626]



In [130]:
def get_quals(file):
    """"""
    with open(file, "r") as f:
        reads = []
        quals = []
        i = 1
        for line in f:
            if (line.startswith("@ERR")) or (line.startswith("+")):
                continue
            if i % 2 != 0:
                reads.append(line.strip())
            if i % 2 == 0:
                quals.append(line.strip())
            i += 1
    return reads, quals


def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip()  # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


read, qual = get_quals("ERR037900_1.first1000.fastq")
seq, quals = readFastq("ERR037900_1.first1000.fastq")

In [131]:
for i in range(len(seq)):
    assert read[i] == seq[i]
for i in range(len(quals)):
    assert qual[i] == quals[i]

In [155]:
def findbadreads(read):
    """Find the bad reads in each line"""

    def mode(lst):
        """return most frequent observation"""

        def getmax(lst):
            """gets index associated with max in list"""
            mx = 0
            for i in lst:
                if i > mx:
                    mx = i
            return lst.index(mx)

        counts = [0] * len(lst)
        for num in lst:
            counts[num] += 1
        return getmax(counts)

    out = []
    for seqs in read:
        if "N" in seqs:
            out.append(seqs.index("N"))
    return mode(out)

In [157]:
findbadreads(read)

66